PART 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import re

In [ ]:
spark = SparkSession.builder.appName("Word_count_HW2").getOrCreate()

In [ ]:
book = spark.sparkContext.textFile("Pinocchio.txt")

In [ ]:
words = book.flatMap(lambda line: re.split(r"\s+", line))
words = words.filter(lambda word: word.strip() != "")

word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts = word_counts.sortBy(lambda x: x[1], ascending=False)

print("every word in the book with their counts:")
for word, count in word_counts.take(20):
    print(f"{word}: {count}")

In [ ]:
clean_words = words.map(lambda word: re.sub(r"[^\w\s]", "", word.lower()))

stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

filtered_words = clean_words.filter(lambda word: word not in stopwords and word != "")

In [ ]:
filtered_word_counts = filtered_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
filtered_word_counts = filtered_word_counts.sortBy(lambda x: x[1], ascending=False)

print("top 15 filtered words with their counts")
for word, count in filtered_word_counts.take(15):
    print(f"{word}: {count}")

PART 2

In [ ]:
spark = SparkSession.builder.appName("Bigram_Count_HW2").getOrCreate()

In [ ]:
book = spark.sparkContext.textFile("Pinocchio.txt")

In [ ]:
words = book.flatMap(lambda line: re.split(r"\s+", line))
words = words.map(lambda word: re.sub(r"[^\w\s]", "", word.lower())).filter(lambda word: word.strip() != "")

stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

filtered_words = words.filter(lambda word: word not in stopwords and word != "")

In [ ]:
bigrams = filtered_words.zipWithIndex().map(lambda x: (x[1], x[0]))
bigrams = bigrams.join(bigrams.map(lambda x: (x[0] - 1, x[1])))
bigrams = bigrams.map(lambda x: (x[1], 1)).filter(lambda x: len(x[0][0]) > 0 and len(x[0][1]) > 0)

In [ ]:

bigrams_counts = bigrams.reduceByKey(lambda a, b: a + b)
bigrams_counts = bigrams_counts.sortBy(lambda x: x[1], ascending=False)

In [ ]:
print("10 most common bigrams from this book:")
for bigram, count in bigrams_counts.take(10):
    print(f"{bigram[0]} {bigram[1]}: {count}")